# ORDER BY Tutorial

## ORDER BY  
**ORDER BY** is usually the last clause in your query, and it sorts the results returned by the rest of the query.  

In [ ]:
query = """
        SELECT ID, Name, Animal
        FROM FROM `bigquery-public-data.pet_records.pets`
        ORDER BY ID  
        """

The **ORDER BY** clause also works for columns containing text, where the results show up in alphabetical order. 

In [ ]:
query = """
        SELECT ID, Name, Animal
        FROM FROM `bigquery-public-data.pet_records.pets`
        ORDER BY Animal  
        """

You can reverse the order using the **DESC** argument. 

In [ ]:
query = """
        SELECT ID, Name, Animal
        FROM FROM `bigquery-public-data.pet_records.pets`
        ORDER BY Animal DESC
        """

## Dates  
There are two ways that dates can be stored in BigQuery: as a **DATE** or as **DATETIME**.  
  
The **DATE** format has the year first, then the month, and then the day. It looks like this:  
`YYYY-[M]M-[D]D`  

* YYYY : Four-digit year  
* [M]M : One or two digit month  
* [D]D : One or two digit day  

The **DATETIME** format is like the date format....but with the time added at the end.  

## EXTRACT  
Often you'll want to look at part of a date, like the year or the day. You can do this with **EXTRACT**.

The query below returns two columns, where column `Day` contains the day corresponding to each entry the `Date` column from the `pets_with_date` table: 

In [ ]:
query="""
      SELECT Name, EXTRACT(DAY from Date) AS Day
      FROM `bigquery-public-data.pet_records.pets_with_date`
      """

SQL is very smart about dates, and we can ask for information beyond just extracting part of the cell. For example, this query returns one column with just the week in the year (between 1 and 53) for each date in the `Date` column.  
  
  

In [ ]:
query="""
      SELECT Name, EXTRACT(WEEK from Date) AS Week
      FROM `bigquery-public-data.pet_records.pets_with_date`
      """

## Example: Which day of the week has the most fatal motor accidents?  
Let's use the US Traffic Fatality Records database, which contains information on traffic accidents in the US where at least one person died.  
We'll investigate the accident_2015 table.   
  
    
We'll use the table to determine how the number of accidents varies with the day of the week:  
Since:  
* the `consecutive_number` column contains a unique `ID` for each accident  
* the `timestamp_of_crash` column contains the date of the accident in DATETIME format,
  
we can:

- **EXTRACT** the day of the week (as `day_of_week` in the query below) from the `timestamp_of_crash` column, and
- **GROUP BY** the day of the week, before we **COUNT** the `consecutive_number` column to determine the number of accidents for each day of the week.  

Then we sort the table with an **ORDER BY** clause, so the days with the most accidents are returned first.


In [ ]:
# Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

As usual, we run it as follows:  

In [ ]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day

## Code Practice  

The value in the indicator_code column describes what type of data is shown in a given row.

One interesting indicator code is `SE.XPD.TOTL.GD.ZS`, which corresponds to "Government expenditure on education as % of GDP (%)".

### 1) Government expenditure on education  
Which countries spend the largest fraction of GDP on education?

To answer this question, consider only the rows in the dataset corresponding to indicator code SE.XPD.TOTL.GD.ZS, and write a query that returns the average value in the value column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average).

Requirements:

* Your results should have the country name rather than the country code. You will have one row for each country.
* The aggregate function for average is AVG(). Use the name avg_ed_spending_pct for the column created by this aggregation.
* Order the results so the countries that spend the largest fraction of GDP on education show up first.                           

In [ ]:
# Your code goes here
country_spend_pct_query = """
                          SELECT country_name, AVG(value) AS avg_ed_spending_pct
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year <= 2017 AND year >= 2010
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

# Check your answer
##q_1.check()

## 2) Identify interesting codes to explore   
The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find more interesting indicator codes to explore?  
  
There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.  
  
Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.  
  
Requirements:  
  
* You should have one row for each indicator code.  
* The columns in your results should be called `indicator_code`, `indicator_name`, and `num_rows`.  
* Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).  
* To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT()** aggregation). This requires you to include both in your **GROUP BY** clause.  
* Order from results most frequent to least frequent.  

In [ ]:
# Your code goes here
code_count_query = """
                   SELECT indicator_code, indicator_name, COUNT(1) AS num_rows
                   FROM `bigquery-public-data.world_bank_intl_education.international_education`
                   WHERE year = 2016
                   GROUP BY indicator_code, indicator_name
                   HAVING COUNT(1) >= 175
                   ORDER BY COUNT(1) DESC
                   """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())

# Check your answer
q_2.check()

## References   

https://www.kaggle.com/dansbecker/order-by  
https://www.kaggle.com/nichollette/exercise-order-by/edit  


--- 